# Day 13 Challenge

In [1]:
import aocd
import session

data = list(map(int, aocd.get_data(session=session.session, year=2019, day=13).strip().split(",")))

In [2]:
import operator
from itertools import chain, cycle, permutations
from collections import deque, defaultdict

class Intcode:
    ops = {1: operator.add,
           2: operator.mul,
           3: 'in',
           4: 'out',
           5: 'jmpif',
           6: 'jmpifn',
           7: lambda i,j: int(i < j),
           8: lambda i,j: int(i == j),
           9: 'rebase',
           99: None,  # signal exit
           }

    def __init__(self, src, inputs=None):
        self.src = defaultdict(int)
        self.src.update(enumerate(src))
        self.ip = 0
        self.base = 0
        self.pipe = None  # potential instance to pipe output into
        if not inputs:
            inputs = deque([])
        self.inputs = deque(inputs)
        self.outputs = []
        self.last_op = None

    def pipe_into(self, other):
        self.pipe = other

    @staticmethod
    def modes_from_op(opval):
        """Decode parameter modes into binary ints"""
        parammodes = chain(map(int, reversed(str(opval//100))), cycle([0])) # 1 for value, 0 for pointer
        return parammodes

    def step(self):
        """Take a step in the intcode program"""
        src = self.src
        ip = self.ip
    
        opval = src[ip]
    
        opcode = opval % 100
        if opcode == 99:
            n_inps = 0
            n_outs = 0
        elif opcode in [1, 2]:
            n_inps = 2
            n_outs = 1
        elif opcode == 3:
            n_inps = 0
            n_outs = 1
        elif opcode == 4:
            n_inps = 1
            n_outs = 0
        elif opcode in [5, 6]:
            n_inps = 2
            n_outs = 0
        elif opcode in [7, 8]:
            n_inps = 2
            n_outs = 1
        elif opcode == 9:
            n_inps = 1
            n_outs = 0
        else:
            assert False, f'Invalid opcode {opcode} at position {i}!'
    
        op = self.ops.get(opcode, -1)
        assert op != -1, f'Need to implement more operations in ops dict!'

        self.last_op = op
    
        parammodes = self.modes_from_op(opval)
        params = []
        for k,mode in zip(range(ip + 1, ip + 1 + n_inps), parammodes):
            if mode == 0:
                index = src[k]
            elif mode == 1:
                index = k
            elif mode == 2:
                index = self.base + src[k]
            else:
                assert False, f'Invalid parameter mode {mode}!'
            assert index >= 0, f'Invalid input index {index}!'

            param = src[index]
            params.append(param)

        if n_outs:
            output_mode = next(parammodes)
            if output_mode == 0:
                ip_out = src[ip + 1 + n_inps]
            elif output_mode == 2:
                ip_out = self.base + src[ip + 1 + n_inps]
            else:
                assert False, 'What does output_mode == 1 mean?'

            assert ip_out >= 0, f'Invalid output index {ip_out}!'
        else:
            ip_out = None

        # handle "input"
        if op == "in":
            print(ip_out)
            print(self.inputs.popleft())
            src[ip_out] = self.inputs.popleft()

        # handle relative base offset
        if op == 'rebase':
            offset = params[0]
            self.base += offset

        # handle jumps, increment intruction pointer
        if (op == 'jmpif' and params[0]) or (op == 'jmpifn' and not params[0]):
            self.ip = params[1]
        else:
            self.ip += 1 + n_inps + n_outs
    
        # handle "output"
        if op == "out":
            # value is params[0]
            res = params[0]
            self.outputs.append(res)
            if self.pipe:
                self.pipe.inputs.append(res)
    
        # compute values for callables
        if callable(op):
            src[ip_out] = op(*params)

        return

def simulate(src, part1=True):
    instance = Intcode(src)
    board = defaultdict(int)
    pos = (0, 0)
    dir = (-1, 0)

    if not part1:
        board[pos] = 1

    while True:
        # loop over pixels
        instance.inputs.append(board[pos])

        # loop over instruction steps until two outputs or exit
        while len(instance.outputs) < 2:
            instance.step()

            if instance.last_op is None:
                # we're done
                if part1:
                    # return colored pixels
                    return len(board)
                else:
                    # return picture
                    print_picture(board)
                    return

        pixel, dirval = instance.outputs
        instance.outputs.clear()

        board[pos] = pixel  # paint
        dir = (dir[1], -dir[0]) if dirval else (-dir[1], dir[0])  # rotate
        pos = pos[0] + dir[0], pos[1] + dir[1]  # step

## Part 1

In [3]:
instance = Intcode(data)

while True:
    instance.step()

384


IndexError: pop from an empty deque

In [4]:
from numpy import array
output = array(instance.outputs).reshape((int(len(instance.outputs)/3)), 3)

count = 0
for element in output:
    if element[2] ==2:
        count += 1
print(f"The number of block (2) elements is {count}.")

The number of block (2) elements is 213.


## Part 2

In [7]:
import numpy as np
import operator
from itertools import chain, cycle, permutations
from collections import deque, defaultdict

class Intcode:
    ops = {1: operator.add,
           2: operator.mul,
           3: 'in',
           4: 'out',
           5: 'jmpif',
           6: 'jmpifn',
           7: lambda i,j: int(i < j),
           8: lambda i,j: int(i == j),
           9: 'rebase',
           99: None,  # signal exit
           }

    def __init__(self, src, inputs=None):
        self.src = defaultdict(int)
        self.src.update(enumerate(src))
        self.ip = 0
        self.base = 0
        self.pipe = None  # potential instance to pipe output into
        if not inputs:
            inputs = deque([])
        self.inputs = deque(inputs)
        self.outputs = []
        self.last_op = None

    def export_state(self):
        return self.src, self.ip, self.base, self.inputs, self.outputs, self.last_op

    def import_state(self, state):
        self.src, self.ip, self.base, self.inputs, self.outputs, self.last_op = state

    def pipe_into(self, other):
        self.pipe = other

    @staticmethod
    def modes_from_op(opval):
        """Decode parameter modes into binary ints"""
        parammodes = chain(map(int, reversed(str(opval//100))), cycle([0])) # 1 for value, 0 for pointer
        return parammodes

    @property
    def next_op(self):
        """Peek at the next input instruction"""
        return self.ops[self.src[self.ip] % 100]

    def step(self):
        """Take a step in the intcode program"""
        src = self.src
        ip = self.ip
    
        opval = src[ip]
    
        opcode = opval % 100
        if opcode == 99:
            n_inps = 0
            n_outs = 0
        elif opcode in [1, 2]:
            n_inps = 2
            n_outs = 1
        elif opcode == 3:
            n_inps = 0
            n_outs = 1
        elif opcode == 4:
            n_inps = 1
            n_outs = 0
        elif opcode in [5, 6]:
            n_inps = 2
            n_outs = 0
        elif opcode in [7, 8]:
            n_inps = 2
            n_outs = 1
        elif opcode == 9:
            n_inps = 1
            n_outs = 0
        else:
            assert False, f'Invalid opcode {opcode} at position {i}!'
    
        op = self.ops.get(opcode, -1)
        assert op != -1, f'Need to implement more operations in ops dict!'

        self.last_op = op
    
        parammodes = self.modes_from_op(opval)
        params = []
        for k,mode in zip(range(ip + 1, ip + 1 + n_inps), parammodes):
            if mode == 0:
                index = src[k]
            elif mode == 1:
                index = k
            elif mode == 2:
                index = self.base + src[k]
            else:
                assert False, f'Invalid parameter mode {mode}!'
            assert index >= 0, f'Invalid input index {index}!'

            param = src[index]
            params.append(param)

        if n_outs:
            output_mode = next(parammodes)
            if output_mode == 0:
                ip_out = src[ip + 1 + n_inps]
            elif output_mode == 2:
                ip_out = self.base + src[ip + 1 + n_inps]
            else:
                assert False, 'What does output_mode == 1 mean?'

            assert ip_out >= 0, f'Invalid output index {ip_out}!'
        else:
            ip_out = None

        # handle "input"
        if op == "in":
            src[ip_out] = self.inputs.popleft()

        # handle relative base offset
        if op == 'rebase':
            offset = params[0]
            self.base += offset

        # handle jumps, increment intruction pointer
        if (op == 'jmpif' and params[0]) or (op == 'jmpifn' and not params[0]):
            self.ip = params[1]
        else:
            self.ip += 1 + n_inps + n_outs
    
        # handle "output"
        if op == "out":
            # value is params[0]
            res = params[0]
            self.outputs.append(res)
            if self.pipe:
                self.pipe.inputs.append(res)
    
        # compute values for callables
        if callable(op):
            src[ip_out] = op(*params)

        return

class Solver:
    def __init__(self, src):
        self.src = src
        self.choices = []  # correct inputs to the intcode program
        self.score = 0
        self.total_blocks = None  # number of initial blocks; part 1
        self.blocks_left = -1  # countdown until end of game
        self.idle_steps = 10  # incremented steps to stand in one place for the next move
        self.checkpoint = None  # state of the interpreter in a safe state
        self.steps = 0  # steps taken for checkpoint
        self.paddley = None  # constant y position of the paddle

    def find_next_choice(self):
        # first stand still, check where ball exits
        instance = Intcode(self.src)
        if self.checkpoint:
            # load state from checkpoint
            instance.import_state(self.checkpoint)

        # but always override inputs (checkpoint inputs are noisy due to idle steps)
        instance.inputs.clear()
        instance.inputs.extend(self.choices + [0]*self.idle_steps)

        steps = self.steps
        while True:
            try:
                instance.step()
            except IndexError:
                # retry with more steps of standing still
                self.idle_steps *= 10
                return
            
            if instance.last_op is None:
                break

            if instance.last_op == 'in':
                steps += 1

                if steps == 1:
                    # count number of initial blocks
                    self.total_blocks = count_blocks(instance.outputs)

                    # assumption: paddle y component is constant, store that too
                    self.paddley = next(y for t,y,x in zip(*[reversed(instance.outputs)]*3) if t == 3)
                elif steps == len(self.choices):
                    # we are guaranteed to hit the paddle, checkpoint here
                    self.checkpoint = instance.export_state()
                    self.steps = steps


        self.blocks_left = count_blocks(instance.outputs)
        if not self.blocks_left:
            # then we've won, need final score
            self.score = next(s for s,y,x in zip(*[reversed(instance.outputs)]*3) if (x,y) == (-1,0))
            return

        # find where the paddle was when we died
        paddlex = next(x for t,y,x in zip(*[reversed(instance.outputs)]*3) if t == 3)

        # find where the ball was two steps ago
        ballx = next(x for t,y,x in zip(*[reversed(instance.outputs)]*3) if (t,y) == (4, self.paddley - 1))

        diff = ballx - paddlex
        sign = np.sign(diff)
        buffers = steps - 1 - len(self.choices) - abs(diff)

        # pad with zeros, then take the number of required steps
        self.choices.extend([0] * buffers + [sign] * abs(diff))
        
        return

def count_blocks(outputs):
    board = {(x,y):t for x,y,t in zip(*[iter(outputs)]*3) if (x,y) != (-1,0)}
    return sum(1 for typ in board.values() if typ == 2)

def print_board(outputs):
    mapping = np.array([' ', '=', '#', '\N{em dash}', 'o'])
    board = {(x,y):t for x,y,t in zip(*[iter(outputs)]*3) if (x,y) != (-1,0)}
    points = np.array(list(board.keys()))
    size = points.ptp(0) + 1
    mins = points.min(0)
    pixels = np.zeros(size, dtype='U1')
    pixels[tuple(points.T)] = mapping[list(board.values())]  # assume ordered dicts
    print('\n'.join([''.join([c for c in row]) for row in np.rot90(pixels, -1).astype(str)]))

def day13(inp):
    src = inp
    src[0] = 2

    solver = Solver(src)
    while solver.blocks_left:
        solver.find_next_choice()

    return solver.total_blocks, solver.score


In [ ]:
day13(data)